In [ ]:
# #Get the required libraries

import pandas as pd 
import re #regular expression  
import string 
 

In [ ]:
## import source of data 
import pandas as pd  

path = "/content/T5_TEXT.txt"
''' to read text file '''
with open(path, 'rb') as f:
    python_string = f.read()
python_string = python_string.decode("utf-8")

## PreProcessing [link text](https://)

In [ ]:
# DATA CLEANING 
import string 
def clean_data(original_text):
    modified_text=original_text.lower()
    modified_text=re.sub ('[\t\n\r\f\v]','',modified_text)
    modified_text=re.sub ('[()[\]{}]','',modified_text)
    modified_text=re.sub('\[\d+\]','',modified_text)
    ''' delat arabic words '''
    modified_text = re.sub('[^a-zA-z0-9@-_.% "]', '',modified_text)
    return modified_text

In [ ]:
OUTPUT=clean_data(python_string)
OUTPUT

### KeyWordsExtraction using YAKE

In [ ]:
!pip install pywsd

In [ ]:
! pip install yake

In [ ]:
import yake
keywords_FINAL=[]
kw_extractor = yake.KeywordExtractor()
#text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(OUTPUT)
for kw in keywords:
    keywords_FINAL.append(kw[0])
print(f'keywords,{keywords_FINAL}')

In [ ]:
keywords_FINAL

## sentence Mapping 
(https://colab.research.google.com/github/Raja-mishra1/Mcq-s-with-Bert-summariser/blob/master/GenerateMCQwithBERT_summarizer_wordnet_and_conceptnet_.ipynb)

In [ ]:
!pip install -U nltk

In [ ]:
! pip install flashtext

In [ ]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import nltk
nltk.download('punkt')


def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    
    # Remove any short sentences less than 20 letters.
    #sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(OUTPUT)
keyword_sentence_mapping = get_sentences_for_keyword(keyphrases, sentences)
        
print (f'sentences{sentences},/n keyword_sentence_mapping {keyword_sentence_mapping}')
len(sentences)

In [ ]:
len(keyphrases)

In [ ]:
keyword_sentence_mapping

In [ ]:
# delet non valuable keys
for k in keyword_sentence_mapping.keys():
  if keyword_sentence_mapping[k]==[]:
    #del(keyword_sentence_mapping[k])
    print(k)


In [ ]:
del(keyword_sentence_mapping["buy products based"
])    


In [ ]:
keyword_sentence_mapping

## delet duplicated sentences

In [ ]:
for key in keyword_sentence_mapping.keys():
  values=keyword_sentence_mapping[key] 
  df = pd.DataFrame({'values':values})
  df.drop_duplicates(inplace = True)
  keyword_sentence_mapping[key] = df['values'].tolist() 

In [ ]:
len(keyword_sentence_mapping)

## creat Questions using T5
https://drive.google.com/drive/folders/1aDic552-3JkGD8-v298tPMoaIvByLPJ4?usp=sharing

> Indented block



In [ ]:
!pip install --quiet git+https://github.com/huggingface/transformers.git@5c00918681d6b4027701eb46cea8f795da0d4064
!pip install --quiet sentencepiece==0.1.95

In [ ]:
!pip install --quiet ipython-autotime
%load_ext autotime

In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer

#T5 model size on disk ~ 900 MB
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

In [ ]:
def get_question(sentence,answer,mdl,tknizer):
  text = "context: {} answer: {}".format(sentence,answer)
  #print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question


#context = "consumers buy products in smaller volumes suitable for personal use."
#answer = "business"

#ques = get_question(context,answer,question_model,question_tokenizer)
#print ("question: ",ques)

In [ ]:
QUESTIONS=[]
CONTEXT_LIST=[]
for k in keyword_sentence_mapping.keys():
  answer=k
  value=keyword_sentence_mapping[k]
  
  context = value[0]
  ques = get_question(context,answer,question_model,question_tokenizer)
  QUESTIONS.append(ques)
  CONTEXT_LIST.append(context)
  #print ("question: ",ques)
  #print(answer)


In [ ]:

# x=list(keyword_sentence_mapping.values())
# x
QUESTIONS

In [ ]:
len( QUESTIONS)
# CONTEXT_LIST


##ALLENNLP

In [ ]:
pip install allennlp==2.1.0 allennlp-models==2.1.0


In [ ]:
!echo '{"passage": "The Matrix is a 1999 science fiction action film written and directed by The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano.", "question": "Who stars in The Matrix?"}' | \
    allennlp predict https://storage.googleapis.com/allennlp-public-models/bidaf-elmo.2021-02-11.tar.gz -

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo.2021-02-11.tar.gz")


In [ ]:
for n in range(len(CONTEXT_LIST)):

  pre=predictor.predict(
  passage=CONTEXT_LIST[n],question=QUESTIONS[n])
  for k in keyword_sentence_mapping.keys():
    print("original answer:",k)
    print("predicted answer:",pre['best_span_str'])




## Another keyWords Extraction

In [ ]:
## install packages
!pip install trafilatura
!pip install summa
!pip install git+https://github.com/smirnov-am/pytopicrank.git#egg=pytopicrank
!pip install git+https://github.com/LIAAD/yake
!pip install keyBERT


Keywords Extraction with BERT




In [ ]:
#from transformers.modeling_bert import BertModel, BertForMaskedLM

from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
for j in range(len(keyword_sentence_mapping)):
   
    keywords = kw_extractor.extract_keywords(keyword_sentence_mapping[j], keyphrase_length=1, stop_words='english')
    for l in keyword_sentence_mapping.keys():
     #print("keywords_bert:",keywords)
      print("yake_keywords_bert:",l)
      print("keywords_bert", str(j+1), "\n", keywords)

### PKE
https://boudinfl.github.io/pke/build/html/tutorials/input.html

## founded that it's the mostaccurate extractor 

## source code _repo
https://github.com/ramsrigouthamg/pke

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
  !python -m nltk.downloader stopwords
  !python -m nltk.downloader universal_tagset
  !python -m spacy download en # download the english model

In [ ]:
import pke

# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

# load the content of the document, here document is expected to be in raw
# format (i.e. a simple text file) and preprocessing is carried out using spacy
extractor.load_document(input='T5_TEXT.txt', language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=20)

In [ ]:
keyphrases = [ seq[0] for seq in keyphrases ]

In [ ]:
keyphrases

### Distractor-Generation-RACE

In [ ]:
# download glive 
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip


In [ ]:
!unzip glove*.zip


In [ ]:
!wget https://github.com/Yifan-Gao/Distractor-Generation-RACE/blob/master/data/data.tar.gz

In [ ]:
!unzip data.tar.gz